In [2]:
import pandas as pd
import numpy as np
import os

`2019-3-18 01:00:00` to `2020-3-18 13:00:00` <br>
Temp, Wind, PM of test set is already cleaned. But Fire spot.

In [9]:
provinces = ['Bangkok','Chanthaburi','Chiang Mai','Kanchanaburi','Khon Kaen','Songkhla']
data = {}
for province in provinces:
    if province=='Khon Kaen' : continue
    df = pd.read_csv(f'./data/Test/{province}_imputed.csv', parse_dates=True, index_col=0)
    data[province] = df
    print(f'{df.index[0]} to {df.index[-1]} ')

2019-03-18 08:00:00 to 2020-03-18 20:00:00 
2019-03-18 08:00:00 to 2020-03-18 20:00:00 
2019-03-18 08:00:00 to 2020-03-18 20:00:00 
2019-03-18 08:00:00 to 2020-03-18 20:00:00 
2019-03-18 08:00:00 to 2020-03-18 20:00:00 


เริ่มและจบที่เวลาเดียวกันทั้งหมด

In [10]:
data['Bangkok']

,PM2.5,Temp(C),WindDir,Wind Speed(km/h)
2019-03-18 08:00:00,45.6,29.3,130.0,17.0
2019-03-18 09:00:00,46.6,29.3,130.0,17.0
2019-03-18 10:00:00,47.7,33.4,120.0,14.0
2019-03-18 11:00:00,43.7,33.4,120.0,14.0
2019-03-18 12:00:00,41.6,33.4,120.0,14.0
...,...,...,...,...
2020-03-18 16:00:00,23.9,33.0,210.0,7.0
2020-03-18 17:00:00,24.4,33.0,210.0,7.0
2020-03-18 18:00:00,24.9,33.0,210.0,7.0
2020-03-18 19:00:00,27.0,30.4,260.0,6.0


ข้อมูล Fire spot ที่ชัดเจนมีถึงแค่สิ้นปี 2019 เราต้องหาจากแหล่งอื่นอีก 3 เดือน (2020, Jan - Mar)

In [14]:
countries = ['Cambodia','Myanmar','Thailand','Lao_PDR']
fire = {}
for country in countries:
    df = pd.read_csv(f'./Fire hotspot/viirs-snpp_2019_{country}.csv')
    df['hour'] = df['acq_time'].apply(str).str[:-2]
    
    df['datetime'] = pd.to_datetime(df['acq_date']+'T'+df['hour'].str.zfill(2), 
                        format='%Y-%m-%dT%H')
    df = df.groupby('datetime').mean()
    # เอาแค่ col ที่สนใจของตาราง fire spot
    # Assume : frp represent brightness
    fire[country] = df[['frp']].rename(columns={'frp':country+'_frp'})

In [18]:
for c in countries:
    print(f"{fire[c].index[0]} to {fire[c].index[-1]}")
    print(len(fire[c]))

2019-01-01 05:00:00 to 2019-12-31 18:00:00
180640
2019-01-01 05:00:00 to 2019-12-31 19:00:00
377443
2019-01-01 05:00:00 to 2019-12-31 19:00:00
232841
2019-01-01 05:00:00 to 2019-12-31 18:00:00
172355


In [17]:
pd.concat(fire)

Cambodia_frp  Myanmar_frp  Thailand_frp  \
         datetime                                                       
Cambodia 2019-01-01 05:00:00          4.79          NaN           NaN   
         2019-01-01 05:00:00          3.31          NaN           NaN   
         2019-01-01 05:00:00          4.79          NaN           NaN   
         2019-01-01 05:00:00         12.62          NaN           NaN   
         2019-01-01 05:00:00         11.68          NaN           NaN   
...                                    ...          ...           ...   
Lao_PDR  2019-12-31 18:00:00           NaN          NaN           NaN   
         2019-12-31 18:00:00           NaN          NaN           NaN   
         2019-12-31 18:00:00           NaN          NaN           NaN   
         2019-12-31 18:00:00           NaN          NaN           NaN   
         2019-12-31 18:00:00           NaN          NaN           NaN   

                              Lao_PDR_frp  
         datetime                          
Cambodia 2019-01-01 05:00:00          NaN  
         2019-01-01 05:00:00          NaN  
         2019-01-01 05:00:00          NaN  
         2019-01-01 05:00:00          NaN  
         2019-01-01 05:00:00          NaN  
...                                   ...  
Lao_PDR  2019-12-31 18:00:00         0.64  
         2019-12-31 18:00:00         2.04  
         2019-12-31 18:00:00         1.06  
         2019-12-31 18:00:00         0.55  
         2019-12-31 18:00:00         0.67  

[963279 rows x 4 columns]